In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment

# Load the Excel file
file_path = '/Users/jacksonsiemens/Desktop/R/Basebol/data/orioles_lineups.xlsx'
player_data = pd.read_excel(file_path)


#Removing players who won't factor into optimization. We are removing them to save on Computation. We will also remove Adley, as his optimal position is always obvious.
players_to_remove = ["Aaron Hicks", "Kyle Stowers", "Adam Frazier", "Heston Kjerstad","Connor Norby", 
                     "James McCann", "Adley Rutschman" ]  
# We will then optimize for 8 positions, and where we can select up to 11 players. This will be more efficient then trying redundant lineups

# Filter the DataFrame to exclude these players
player_data = player_data[~player_data['Name'].isin(players_to_remove)]
player_data = player_data.reset_index(drop=True)
player_data


,id,Name,bats,bat,bat_sd,plat,bat_vsRHP,bat_vsLHP,C,1B,2B,3B,SS,LF,CF,RF
0,BAL,Cedric Mullins,L,3.7,19,18,12.7,-5.3,NaN,NaN,NaN,NaN,NaN,15.0,8.0,13.0
1,BAL,Austin Hays,R,1.3,9,-8,-2.7,5.3,NaN,NaN,NaN,NaN,NaN,6.0,-4.0,8.0
2,BAL,Colton Cowser,L,0.3,11,18,9.3,-8.7,NaN,NaN,NaN,NaN,NaN,7.0,1.0,5.0
3,BAL,Ryan McKenna,R,-1.0,21,-3,-2.5,0.5,NaN,NaN,NaN,NaN,NaN,7.0,3.0,4.0
4,BAL,Hudson Haskin,R,0.0,21,-6,-3.0,3.0,NaN,NaN,NaN,NaN,NaN,4.0,1.0,2.0
5,BAL,Sam Hilliard,L,-1.0,9,18,8.0,-10.0,NaN,NaN,NaN,NaN,NaN,0.0,-4.0,2.0
6,BAL,Jorge Mateo,R,-1.8,20,-8,-5.8,2.2,NaN,NaN,9.0,4.0,13.0,2.0,-3.0,0.0
7,BAL,Terrin Vavra,R,0.0,10,18,9.0,-9.0,NaN,NaN,-5.0,-8.0,NaN,2.0,-9.0,-1.0
8,BAL,Anthony Santander,S,10.5,14,7,14.0,7.0,NaN,-4.0,NaN,NaN,NaN,-8.0,NaN,-10.0
9,BAL,Ryan O'Hearn,L,1.2,22,18,10.2,-7.8,NaN,-3.0,NaN,NaN,NaN,-6.0,NaN,-11.0


In [10]:
import numpy as np
from scipy.optimize import linear_sum_assignment

player_data.fillna(-1000, inplace=True)

def optimize_dual_lineups(player_data):
    positions = ['1B', '2B', '3B', 'SS', 'LF', 'CF', 'RF']
    max_players = 11

    # Calculate weighted offensive RAA for each player
    player_data['weighted_offensive_RAA'] = (player_data['bat_vsRHP'] * 0.7299 + player_data['bat_vsLHP'] * 0.2701)

    # Function to create an optimized lineup
    def create_optimized_lineup(pitcher_type):
        bat_column = f'bat_vs{pitcher_type}'
        total_performance_matrix = np.full((len(player_data), len(positions) + 1), -1000.0)  # Include DH

        for i, position in enumerate(positions):
            can_play = player_data[position] > -1000
            total_performance_matrix[can_play, i] = player_data[bat_column][can_play] + player_data[position][can_play]

        total_performance_matrix[:, -1] = player_data[bat_column]

        row_ind, col_ind = linear_sum_assignment(-total_performance_matrix)
        optimized_runs = total_performance_matrix[row_ind, col_ind].sum()
        optimized_lineup = player_data.iloc[row_ind]
        positions_with_dh = positions + ['DH']
        optimized_lineup['Position'] = [positions_with_dh[i] for i in col_ind]

        return optimized_lineup, optimized_runs, set(row_ind)

    # Optimize the lineup against RHP and LHP
    rhp_lineup, rhp_runs, rhp_players = create_optimized_lineup('RHP')
    lhp_lineup, lhp_runs, lhp_players = create_optimized_lineup('LHP')

    # Combine players and adjust if more than max_players
    combined_players = rhp_players.union(lhp_players)
    while len(combined_players) > max_players:
        # Adjust logic to reduce player count
        least_contributing_player = min(combined_players - rhp_players, key=lambda x: player_data.iloc[x]['weighted_offensive_RAA'])
        player_data.at[least_contributing_player, 'weighted_offensive_RAA'] = -1000
        rhp_lineup, rhp_runs, rhp_players = create_optimized_lineup('RHP')
        lhp_lineup, lhp_runs, lhp_players = create_optimized_lineup('LHP')
        combined_players = rhp_players.union(lhp_players)

    return rhp_lineup, rhp_runs, lhp_lineup, lhp_runs, list(combined_players)

# Example usage:
rhp_lineup, rhp_runs, lhp_lineup, lhp_runs, player_list = optimize_dual_lineups(player_data)


/var/folders/8n/g8bq8_011kv24tnfb649vgmw0000gn/T/ipykernel_12181/2077293236.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  optimized_lineup['Position'] = [positions_with_dh[i] for i in col_ind]
/var/folders/8n/g8bq8_011kv24tnfb649vgmw0000gn/T/ipykernel_12181/2077293236.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  optimized_lineup['Position'] = [positions_with_dh[i] for i in col_ind]
/var/folders/8n/g8bq8_011kv24tnfb649vgmw0000gn/T/ipykernel_12181/2077293236.py:28: SettingWithCopyWarning: 
A va

KeyboardInterrupt: 